# Noise reduction on event data

In this notebook , we will demonstrate the application of both common and event-specific noise reduction techniques. We will vizualize some results and compare different methods to maximize the effectiveness of the noise reduction feature, which is specifically designed for event data.

<br>

***